# Assessment Problems

## Imports

In [127]:
# NumPy.
import numpy as np
from typing import Generator 

# Haslib for Problem 4 & Problem 5
import hashlib 

## Problem 1: Binary Words and Operations

### Technical Glossary
This section defines operations & technical terms used throughout the implementation of solutions for the problems set out in the assignment.

### Bitwise Logical Operations

**Bitwise Operations** are operations that work on the individiual bits of a binary number. Applying the operator to each bit individually but all bits simulatenously.

**XOR (Exclusive OR)**
Outputs 1 when the input bits are different.

| A | B | A ⊕ B|
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   1   |
| 1 | 0 |   1   |
| 1 | 1 |   0   |

**AND**
Outputs 1 when both input bits are 1

| A | B | A & B|
|---|---|-------|
| 0 | 0 |   0   |
| 0 | 1 |   0   |
| 1 | 0 |   0   |
| 1 | 1 |   1   |

**NOT**
Output flips the bits

| A | ¬A|
|---|---|
|0  |1  |
|1  |0  |

**Bit Manipulation Operations**
* `ROTR^n(x)`: Rotate right - shifts bits to the right n times, 'fallen' bits move back to the left (start)
* `SHR^n(x)`: Logical shift right - shifts bits to the right n times; fills left with zeros

**Common Terms**
* **Bitwise Operation:** Performs an operation on each individual bit
* **32-bit unsigned integer:** An integer represented by 32 individual bits in a binary sequence. "Unsigned" meaning non-negative values.
* **Diffusion:** Where the input changes are spread throughout the output

See: [FIPS 180-4, Sections 2.2.2 and 3.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

### Implementing Parity
**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Parity}(x, y, z) = x \oplus y \oplus z$

**How it works:**
Chains $'\oplus'$ operations across three 32-bit integers.

Since $'\oplus'$ is a binary operation it takes 2 inputs. The expression is evaluated as $ \text (x \oplus y \oplus z)$
. The result of $x \oplus y$ is then, result $\oplus z$.

Each output bit is 1 if there is an odd number of input bits set to 1, otherwise 0. This is why its called "Parity" it checks the parity(odd/even count) of bits at each position.

**Example**

X = 1100

Y = 1100

Z = 0110

Output = 0110

In [128]:
def parity(x, y, z) -> np.uint32:
    """
    Calculate the parity (bitwise XOR) of three 32-bit unsigned integers.

    Args:
        x, y, z (int): 32-bit unsigned integer.

    Returns:
        int: The parity of the three integers as a 32-bit unsigned integer.

    """
    
    # Loop over each passed parameters
    for args in (x, y, z):
        # Use isInstance to ensure correct type (object, desired type)
        # See: https://www.w3schools.com/python/ref_func_isinstance.asp
        if not isinstance(args, (int, np.integer)):
            # Raises / Throws a TypeError with a message
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast all inputs as NumPy unsigned 32-bit integers.
    # See: https://numpy.org/doc/stable/reference/arrays.scalars.html#numpy.uint32
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Apply bitwise XOR (^) to the three inputs and return
    return x ^ y ^ z

In [129]:
def testing_parity():
    assert parity(1, 0, 0) == np.uint32(1)
    assert parity(1, 1, 0) == np.uint32(0)
    assert parity(7, 13, 5) == np.uint32(15)

def test_error_on_parity():
    try:
        parity(1.5, 2, 3)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_parity()
test_error_on_parity()

print("All test cases pass on Parity")

All test cases pass on Parity


### Implementing Choose (Ch)
**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Ch}(x, y, z) = (x \land y) \oplus (\lnot x  \land  z)$

__How it works:__
The $\text{Ch}(x, y, z)$ uses x as a selector mask. For each bit position, if x is 1, the output takes the bit from y.
If x is 0, the output takes the bit from z.

This is achieved by calculating $\text (x \land y)$ to select bits from y where x is 1, then $\text (\lnot x \land y)$ to select bits from z where x is 0. Then $'\oplus'$ the results together.

**Example**

X = 1110 

Y = 1010 

Z = 1111 

Output = 1011

In [130]:
def ch(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Choose: return bits from y where x is 1, and bits from z where x is 0.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'masked' value of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over the parameters passed
    for args in (x, y, z):
        # Use isinstance + raise TypeError as before
        # See: Problem 1 -> Implementing Parity
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")

    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation and return the value
    return (x & y) ^ (~x & z)

In [131]:
def testing_ch():
    assert ch(1, 0, 0) == np.uint32(0)
    assert ch(1, 3, 6) == np.uint32(7)
    assert ch(3, 5, 8) == np.uint32(9)

def test_error_on_ch():
    try:
        ch(2.5, 4, 9)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_ch()
test_error_on_ch()

print("All test cases pass on Ch")

All test cases pass on Ch


### Implementing Maj
**Purpose:** The Maj computes the bitwise (XOR, AND) of 3 unsigned 32-bit integers. 

**Spec:** Defined in the Secure Hash Standard [Page 10](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf) as:

> $\text{Maj}(x, y, z) = (x \land y) \oplus (x \land  z) \oplus (y \land  z)$

**How it works:**
The $\text{Maj}(x, y, z)$ implements majority voting at each bit position. For each bit, the output is 1 if at least two input bits are 1, otherwise 0.
This is computed by taking the $'\land'$ of each pair of inputs 
$\text (x \land y)$, $\text (x \land z)$, and $\text (y \land z)$, then $'\oplus'$ the three results together.

**Example:**

X = 11101110

Y = 10100101

Z = 11111001

Output = 11101101

In [132]:
def maj(x: int, y: int, z: int) -> np.uint32:
    """
    Bitwise Majority: output bit is 1 if two or more input bits are 1.

    Parameters
    ----------
        x, y, z : int
            32-bit unsigned integers.

    Returns
    -------
        numpy.uint32
            The output 'voted' values of the three integers.

    Raises
    ------
        TypeError
            If any argument is not an integer.
    """

    # Loop over args, use isinstance & raise TypeError as before
    # See: Problem 1 -> Implementing Parity
    for args in (x, y, z):
        if not isinstance(args, (int, np.integer)):
            raise TypeError("All arguments must be integers. No other types are allowed.")
    
    # Cast values as before, see: Problem 1 -> Implementing Parity
    x, y, z = np.uint32(x), np.uint32(y), np.uint32(z)

    # Perform the bitwise operation & return the value
    return (x & y) ^ (x & z) ^ (y & z)

In [133]:
def testing_maj():
    assert maj(1, 2, 3) == np.uint32(3)
    assert maj(3, 4, 7) == np.uint32(7)
    assert maj(5, 8, 10) == np.uint32(8)

def test_error_on_maj():
    try:
        maj(2, 4.7, 6)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
testing_maj()
test_error_on_maj()

print("All test cases pass for Maj")

All test cases pass for Maj


#### Big Sigma 0 - $\Sigma_0^{\{256\}}(x)$

**Purpose**:
Provides non-linear mixing by combining three rotated versions of the input. This ensures the bits are widely spread across. Increasing diffusion.
    
**Spec.** Defined in the Secure Hash Standard as:
    
> $\Sigma_0^{\{256\}}(x) = \text{ROTR}^{2}(x) \oplus \text{ROTR}^{13}(x) \oplus \text{ROTR}^{22}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**Intuition:**

Rotating the words by different amounts makes sure each bit affect multiple output bit positions.
XOR-ing the results ensures the output is very complex to reverse.

**Example**

x = 10110011

Each bit is rotated **2**, **13** & **22** times, then XOR'ed together.

These numbers are **not** arbitrary they are a result of testing during the creation of the Secure Hash Standard.

They reinforce the avalanche effect & ensure good spread diffusion and mixing.

In [134]:
def big_sigma0(x: int) -> np.uint32:
    """
    Performs the Σ₀ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function 
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input 
    word by 2, 13, and 22 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x), as a 32-bit unsigned integer.
    """
    
    if not (isinstance(x, (int, np.uint32))):
        raise TypeError("Input must be a non-negative integer")
    
    x = np.uint32(x)

    # Perform the circular right rotations 
    rotr2 = np.uint32((x >> 2) | (x << 30)) # 32 - 2 = 30
    rotr13 = np.uint32((x >> 13) | (x << 19)) # 32 - 13 = 19
    rotr22 = np.uint32((x >> 22) | (x << 10)) # 32 - 22 = 10

    return rotr2 ^ rotr13 ^ rotr22

#### Big Sigma 1 - $\Sigma_1^{\{256\}}(x)$

**Purpose**:
Exact same function as Sigma 0, except with different rotation constants, providing another layer of complexity, diffusion and mixing.
    
**Spec.** Defined in the Secure Hash Standard as:
    
> $\Sigma_1^{\{256\}}(x) = \text{ROTR}^{6}(x) \oplus \text{ROTR}^{11}(x) \oplus \text{ROTR}^{25}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

**Intuition:**

Rotating the words by different amounts makes sure each bit affect multiple output bit positions.
XOR-ing the results ensures the output is very complex to reverse.

**Example**

x = 10110011

Each bit is rotated **6**, **11** & **25** times, then XOR'ed together.

These numbers are **not** arbitrary they are a result of testing during the creation of the Secure Hash Standard.

They reinforce the avalanche effect & ensure good spread diffusion and mixing.

In [135]:
def big_sigma1(x: int) -> np.uint32:
    """
    Performs the Σ₁ function on a 32-bit word.

    This function is used in:
    - The SHA-256 compression function
    - The SHA-224 compression function

    The operation involves three circular right rotations (ROTR) of the input
    word by 6, 11, and 25 bits, followed by a bitwise XOR of the results.

    Parameters:
        x (int): A 32-bit word.

    Returns:
        np.uint32: The result of ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x), as a 32-bit unsigned integer.
    """

    if not (isinstance(x, (int, np.uint32))):
        raise TypeError("Input must be a non-negative integer")
    
    x = np.uint32(x)

    # Perform the circular right rotations
    rotr6 = np.uint32((x >> 6) | (x << 26))  # 32 - 6 = 26
    rotr11 = np.uint32((x >> 11) | (x << 21))  # 32 - 11 = 21
    rotr25 = np.uint32((x >> 25) | (x << 7))  # 32 - 25 = 7

    return rotr6 ^ rotr11 ^ rotr25

print(big_sigma1(18))

1245710592


In [136]:
def test_big_sigma_0_valid():
    assert big_sigma0(1)  == 1074267136
    assert big_sigma0(10) == 2152736770

def test_big_sigma_1_valid():
    assert big_sigma1(7)  == 484443008
    assert big_sigma1(18) == 1245710592


def test_big_sigma_type_error():
    try:
        big_sigma0(1.5)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        big_sigma1(1.5)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_big_sigma_0_valid()
test_big_sigma_1_valid()
test_big_sigma_type_error()

print("All test cases pass on Big Sigma 0 & 1")

All test cases pass on Big Sigma 0 & 1


#### Small Sigma 0 - $\sigma_0^{\{256\}}(x)$
* __Small Sigma 0__  -> $'\oplus'$ Bitwise, 2 rotations (7, 18) & Bitwise shift right (3) - SHA-256 Reference required here

**Spec.** Defined in the Secure Hash Standard as:
> $\sigma_0^{\{256\}}(x) = \text{ROTR}^{7}(x) \oplus \text{ROTR}^{18}(x) \oplus \text{SHR}^{3}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [137]:
def small_sigma0(x: int) -> np.uint32:
    """
    Performs the σ₀ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 7 and 18 positions, one right shift (SHR) by 3 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x), as a 32-bit unsigned integer
    """
    # Ensure input is treated as a 32-bit unsigned integer

    if not (isinstance(x, (int, np.uint32))):
        raise TypeError("Input must be a non-negative integer")
    
    x = np.uint32(x)
    
    # Perform circular right rotations by 7 and 18 bits
    rotr7 = np.uint32((x >> 7) | (x << 25)) # 32 - 7 = 25
    rotr18 = np.uint32((x >> 18) | (x << 14)) # 32 - 18 = 14

    # Perform right shift by 3 bits
    # SHR^n(x) = x >> n (fills with zeros on the left)
    shr3 = np.uint32(x >> 3)
    
    # XOR all three values together
    return rotr7 ^ rotr18 ^ shr3

#### Small Signma 1 - $\sigma_1^{\{256\}}(x)$
* __Small Sigma 1__  -> $'\oplus'$ Bitwise, 2 rotations (17, 19) & Bitwise shift right (10) - SHA-256 Reference required here

**Spec.** Defined in the Secure Hash Standard as:
> $\sigma_1^{\{256\}}(x) = \text{ROTR}^{17}(x) \oplus \text{ROTR}^{19}(x) \oplus \text{SHR}^{10}(x)$

See: [FIPS 180-4, Sections 4.1.2](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [138]:
def small_sigma1(x: int) -> np.uint32:
    """
    Performs the σ₁ function on a 32-bit word.

    This function is used in the SHA-256 and SHA-224 message schedule expansion.

    The function performs two circular right rotations (ROTR) of the input
    by 17 and 19 positions, one right shift (SHR) by 10 positions, followed 
    by a bitwise XOR of the results.
    
    Parameters:
        x (int): A 32-bit word
        
    Returns:
        np.uint32: The result of ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x), as a 32-bit unsigned integer
    """

    if not (isinstance(x, (int, np.uint32))):
        raise TypeError("Input must be a non-negative integer")
    
    # Ensure input is treated as a 32-bit unsigned integer
    x = np.uint32(x)
    
    # Perform circular right rotations by 17 and 19 bits
    rotr17 = np.uint32((x >> 17) | (x << 15)) # 32 - 17 = 15
    rotr19 = np.uint32((x >> 19) | (x << 13)) # 32 - 19 = 13

    # Perform right shift by 10 bits
    # SHR^n(x) = x >> n (fills with zeros on the left)
    shr10 = np.uint32(x >> 10)
    
    # XOR all three values together
    return rotr17 ^ rotr19 ^ shr10

print(small_sigma0(10))
print(small_sigma1(12))

335708161
491520


In [139]:
def test_small_sigma_0_valid():
    assert small_sigma0(1)  == 33570816
    assert small_sigma0(10) == 335708161

def test_small_sigma_1_valid():
    assert small_sigma1(5)  == 139264
    assert small_sigma1(12)  == 491520

# Error on this function like using floating numbers
def test_small_sigma_type_error():
    try:
        small_sigma0(-1.75)
    except TypeError:
        pass
    else:
        raise AssertionError("Expected TypeError for float input")
    
    try:
        small_sigma1(5.25)
    except TypeError:
        pass
    else:
         raise AssertionError("Expected TypeError for float input")

test_small_sigma_0_valid()
test_small_sigma_1_valid()
test_small_sigma_type_error()

print("All test cases pass Small Sigma 0 & 1")

All test cases pass Small Sigma 0 & 1


## End of Problem 1

## Problem 2: Fractional Parts of Cube Roots

## Primes Function
**Purpose:** Every integer greater than 1 can be represented uniquely as a product of prime numbers. This could generates the 1st 64 prime numbers. Start with the 1st prime number and all the way to the 64th prime number. 

**Implementing Prime**
* Implementing a function to calculate the first n prime numbers

References:

See: [GeeksForGeeks Implementation Of Calculating Finding N Prime Numbers](https://www.geeksforgeeks.org/dsa/generate-and-print-first-n-prime-numbers/)

**Example:** 
* 2, 3, 5 is a prime number.
* 4, 10, 12 is NOT a prime number because it's a factor number. Like 10 / 2 = 5
* 0, 1 is a Non-Prime number

In [140]:
def primes(n):
    """
    Generates the first n prime numbers using a simple trial division method.
    
    Parameters:
        n (int): The number of prime numbers to generate
        
    Returns:
        list: A list containing the first n prime numbers
    """
    if n <= 0:
        return [] # Return an empty list for non-positive input

    prime_list = [2] # List to store prime numbers starting with the first prime
    candidate = 3 # Start checking for primes from the first odd prime number

    # While loop until we have n primes
    while len(prime_list) < n:
        is_prime = True # Assume candidate is prime until proven otherwise
        for p in prime_list: # Check divisibility with known primes
            if p * p > candidate: 
                break # No need to check further if p^2 exceeds candidate
            if candidate % p == 0:
                is_prime = False # Found a divisor, not prime
                break
        
        if is_prime:
            prime_list.append(candidate) # Add candidate to the list if it's prime
        
        candidate += 2 # Odd numbers only, skip even numbers

    return prime_list # Return the list of prime numbers

In [141]:
# Generate the first 64 prime numbers
first_64_primes = primes(64)

### Testing Prime
* Tests to see if the function can return the first N Prime numbers.
* Ensure the correct errors are being raised for passed args.

## Calculating the Cube Root
**Purpose:** After getting the first 64 prime numbers, collect the first 32 bits of fractional by calculating the cube root

* Code for calculating cube root of first 64 prime numbers per the brief

[For Calculating Cube Roots in Python](https://www.geeksforgeeks.org/python/python-program-for-find-cubic-root-of-a-number/)

**Example:**
> $\text{root} = {prime}^{1/3}$

> $\text{frac} = {root} - {int(root)}$

> $\text{frac} = {frac}({2}^{32})$

In [142]:
sha256_constants = [] # List to store the K constants

for prime in first_64_primes: 
    cube_root = np.cbrt(prime) # Calculate the cube root
    fractional_part = cube_root - np.floor(cube_root) # Extract the fractional part
    bits32 = np.uint32(fractional_part * (2**32)) # Scale to 32 bits
    sha256_constants.append(bits32) # Append to the constants list

In [143]:
# Display constants in hexadecimal format (8 per line)
for i in range(0, 64, 8):
    print(', '.join(f"0x{k:08x}" for k in sha256_constants[i:i+8])) # Display in hex format

0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5
0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174
0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da
0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967
0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85
0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070
0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3
0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2


### Test Cases (Cube Roots)
**Testing:**
* Correct calculation of cube roots for first 64 primes
* Proper hexadecimal formatting (8 characters each)
* Type enforcement for invalid inputs
* Validation against SHA-256 K constants


See: [FIPS 180-4, Section 4.2.2, p.11](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf)

In [144]:
def test_sha256_constants():
    """
    Test that the SHA-256 K constants match the official constants
    as specified in the SHA-256 standard.
    """
    official_constants = [
        0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
        0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
        0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
        0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
        0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
        0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
        0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
        0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    ]

    assert sha256_constants == official_constants, "SHA-256 K constants do not match official values"
    print("Test passed: SHA-256 K constants match the official values.")

# Run the test
test_sha256_constants()

Test passed: SHA-256 K constants match the official values.


## End of Problem 2

## Problem 3: Padding

### Solution:
### Implementing Block Parse
**Purpose:** SHA-256 processes messages in fixed-size blocks of 512 bits (64 bytes). Messages of arbitrary length must be padded to ensure the final length is a multiple of 512 bits. This function implements the padding scheme and yields message blocks one at a time.

**What is padding?** Padding adds extra data to a message to reach a required length. For SHA-256, padding ensures:

1. The message ends on a block boundary (multiple of 512 bits)
2. The original message length is preserved and authenticated
3. Different messages produce different padded results (no collisions from padding alone)

**Specification** (FIPS 180-4, Sections 5.1.1 and 5.2.1):

The padding process:

1. Append a single '1' bit (byte 0x80 = 10000000 in binary)
2. Append '0' bits until message length ≡ 448 mod 512 (leaves 64 bits remaining)
3. Append original message length as 64-bit big-endian integer
4. Split padded message into 512-bit blocks

**Why this padding scheme?**

* The '1' bit ensures there's always at least some padding (prevents ambiguity)
* The length field prevents length-extension attacks
* The specific modulo ensures exactly 64 bits remain for the length

**Example:**

Original message: "abc" (24 bits)

_Step 1_ - Append '1' bit: "abc" + 0x80

_Step 2_ - Append zeros: Pad to 448 bits (56 bytes)

_Step 3_ - Append length: Add 0x0000000000000018 (24 in 64-bit)

_Result:_ Single 512-bit block

**Generator function:** This implementation uses a Python generator, which yields blocks one at a time rather than returning all blocks at once. This is memory-efficient for large messages.

**Jargon explanation:**

* **Generator:** A function that uses yield to produce values one at a time, pausing between each
* **Big-endian:** Most significant byte first (standard network byte order)
* **mod 512:** Remainder when divided by 512

In [145]:
def block_parse(msg):
    """
    Generator function that parses a message into 512-bit blocks with SHA-256 padding.
    
    Implements padding according to sections 5.1.1 and 5.2.1 of the Secure Hash Standard.
    
    Parameters:
        msg (bytes): The input message to be parsed
        
    Yields:
        bytes: Each 512-bit (64-byte) block of the padded message
    """
    # Length of original message in bits
    msg_len_bits = len(msg) * 8
    
    # Convert message to mutable bytearray for padding 
    padded_msg = bytearray(msg)
    
    # Append the '1' bit (10000000 in binary, or 0x80 in hex)
    padded_msg.append(0x80)
    
    # Calculate the length mod 64 to determine padding needed to reach 56 bytes (448 bits)
    padding_mod = len(padded_msg) % 64
    padding_needed = (56 - padding_mod) if padding_mod <= 56 else (120 - padding_mod)
    
    # Append zero bytes to reach the required length before length encoding
    padded_msg.extend([0x00] * padding_needed)
    
    # Append the original message length as a 64-bit big-endian integer
    padded_msg.extend(msg_len_bits.to_bytes(8, byteorder='big'))
    
    # Yield successive 512-bit (64-byte) blocks
    for i in range(0, len(padded_msg), 64):
        yield bytes(padded_msg[i:i+64])

### Test Cases (Padding)
* Correct padding for messages of various lengths (short messages, messages near block boundaries)
* Proper block size validation (all output blocks are exactly 64 bytes/512 bits)
* Type enforcement for non-bytes input and verification against known test vectors

In [146]:
def test_block_parse():
    # Test 1: Empty message
    msg1 = b"" 
    blocks1 = list(block_parse(msg1)) # Convert generator to list for testing
    assert len(blocks1) == 1, "Empty message should produce 1 block"
    expected_block1 = b'\x80' + b'\x00' * 55 + (0).to_bytes(8, 'big') # \x80 is 10000000 in binary, \x00*55 pads to 56 bytes, last 8 bytes are length (0 bits)
    assert blocks1[0] == expected_block1, "Padding for empty message incorrect"

    # Test 2: Short message "abc"
    msg2 = b"abc"
    blocks2 = list(block_parse(msg2))
    assert len(blocks2) == 1, "3-byte message should produce 1 block"
    padding_len = 56 - (len(msg2) + 1)  # +1 for 0x80 byte
    expected_block2 = msg2 + b'\x80' + b'\x00' * padding_len + (len(msg2)*8).to_bytes(8, 'big')
    assert blocks2[0] == expected_block2, "Padding for short message incorrect"

    # Test 3: Boundary case - 55-byte message (one block)
    msg3 = b'a' * 55
    blocks3 = list(block_parse(msg3))
    assert len(blocks3) == 1, "55-byte message should produce 1 block"
    padding_len = 56 - (len(msg3) + 1)
    expected_block3 = msg3 + b'\x80' + b'\x00' * padding_len + (len(msg3)*8).to_bytes(8, 'big')
    assert blocks3[0] == expected_block3, "Padding for 55-byte message incorrect"

    # Test 4: Boundary case - 56-byte message (requires 2 blocks)
    msg4 = b'a' * 56
    blocks4 = list(block_parse(msg4))
    assert len(blocks4) == 2, "56-byte message should produce 2 blocks"

    expected_block4_0 = msg4 + b'\x80' + b'\x00' * 7
    expected_block4_1 = b'\x00' * 56 + (len(msg4)*8).to_bytes(8, 'big')

    assert blocks4[0] == expected_block4_0, "First block for 56-byte message incorrect"
    assert blocks4[1] == expected_block4_1, "Padding block for 56-byte message incorrect"

    # Test 5: Exact 64-byte message (2 blocks)
    msg5 = b'b' * 64
    blocks5 = list(block_parse(msg5))
    assert len(blocks5) == 2, "64-byte message should produce 2 blocks"

    expected_block5_0 = msg5
    expected_block5_1 = b'\x80' + b'\x00' * 55 + (len(msg5)*8).to_bytes(8, 'big')

    assert blocks5[0] == expected_block5_0, "First block for 64-byte message incorrect"
    assert blocks5[1] == expected_block5_1, "Padding block for 64-byte message incorrect"


    print("All block_parse function test cases passed.")

test_block_parse()

All block_parse function test cases passed.


## End of Problem 3

## Problem 4: Hashes

### Instruction
- Write a function `hash(current, block)` that calculates the next hash value given the `current` hash value and the next message `block` according to section *6.2.2 SHA-256 Hash Computation* on page 22 of the Secure Hash Standard.

### Solution:

In [147]:
# Ignore overflow warnings for SHA-256 arithmetic 
np.seterr(over='ignore'); # See https://numpy.org/devdocs/reference/generated/numpy.seterr.html

In [148]:
def hash(current: list[np.uint32], block: bytes) -> list[np.uint32]:
    """
    Calculates the next hash value given the current hash and a 512 bit message block.
    
    Implements SHA-256 hash computation according to section 6.2.2 of the Secure Hash Standard.
    
    Note: I have marked each step of the algorithm with comments corresponding to the steps
    in the standard. See section 6.2.2 steps labeled 1-4 https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf

    Parameters:
        current (list): List of 8 np.uint32 values representing the current hash (H0-H7)
        block (bytes): A 512-bit (64-byte) message block
        
    Returns:
        list: List of 8 np.uint32 values representing the updated hash state
    """

    # Ensure current hash values are 32-bit unsigned integers
    H = [np.uint32(h) for h in current]

    # Step 1: Prepare the message schedule (W[0..63])
    W = []

    # First 16 words come directly from the block (big-endian)
    # int.from_bytes converts 4 bytes to a 32-bit integer
    # See: https://docs.python.org/3/library/stdtypes.html#int.from_bytes
    for i in range(16):
        word = int.from_bytes(block[i*4:(i+1)*4], byteorder='big')
        W.append(np.uint32(word))
    
    # Extend the first 16 words into the remaining 48 words
    # Each new word depends on specific earlier words via the small_sigma functions defined in Problem 1
    for i in range(16, 64):
        s0 = small_sigma0(W[i-15])
        s1 = small_sigma1(W[i-2])
        w = np.uint32(W[i-16])
        w = np.uint32(w + s0)
        w = np.uint32(w + W[i-7])
        w = np.uint32(w + s1)
        W.append(w)


    # Step 2: Initialize working variables from current hash values
    a, b, c, d, e, f, g, h = H

    # Step 3: Perform 64 rounds of compression
    for t in range(64):
        # Calculate the two temporary values T1 and T2 with 32-bit wrapping at each addition
        T1 = np.uint32(h)
        T1 = np.uint32(T1 + big_sigma1(e)) # Add big_sigma1(e) (big_sigma1 function defined in Problem 1)
        T1 = np.uint32(T1 + ch(e, f, g)) # Add ch(e,f,g) (ch function defined in Problem 1)
        T1 = np.uint32(T1 + sha256_constants[t]) # Add sha256_constants[t] (sha256_constants defined in Problem 2)
        T1 = np.uint32(T1 + W[t]) # Add W[t]

        # Calculate T2
        T2 = np.uint32(big_sigma0(a)) # Add big_sigma0(a) (big_sigma0 function defined in Problem 1)
        T2 = np.uint32(T2 + maj(a, b, c)) # Add maj(a,b,c) (maj function defined in Problem 1)

        # Update working variables
        h = g
        g = f
        f = e
        e = np.uint32(d + T1)  # Wrap 32-bit addition
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)  # Wrap 32-bit addition
    
    # Step 4: Compute the next hash value
    next_hash = [
        np.uint32(H[0] + a),
        np.uint32(H[1] + b),
        np.uint32(H[2] + c),
        np.uint32(H[3] + d),
        np.uint32(H[4] + e),
        np.uint32(H[5] + f),
        np.uint32(H[6] + g),
        np.uint32(H[7] + h)
    ]
    
    # Return updated hash values as a list of 8 np.uint32 values
    return next_hash

In [149]:
# Test the hash function
def test_hash():
    # SHA-256 initial hash values (first 32 bits of fractional parts of square roots of first 8 primes)
    # Official values can be found in section 5.3.3 of the Secure Hash Standard
    # https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf
    initial_hash = [
        np.uint32(0x6a09e667),
        np.uint32(0xbb67ae85),
        np.uint32(0x3c6ef372),
        np.uint32(0xa54ff53a),
        np.uint32(0x510e527f),
        np.uint32(0x9b05688c),
        np.uint32(0x1f83d9ab),
        np.uint32(0x5be0cd19)
    ]

    # Test message
    msg = b"Nobody inspects the spammish repetition"

    # Split and pad message into 512-bit blocks
    # Using the block_parse function defined in Problem 3
    blocks = list(block_parse(msg))

    # Process each block in sequence
    current_hash = initial_hash[:]
    for block in blocks:
        current_hash = hash(current_hash, block)

    # Produce final digest (concatenate hex values)
    result = ''.join(f'{h:08x}' for h in current_hash)

    # Use hashlib to compute expected SHA-256 hash for verification
    # .sha256() creates a new sha256 hash object
    # .hexdigest() returns the digest as a hexadecimal string
    # See: https://docs.python.org/3/library/hashlib.html
    expected = hashlib.sha256(msg).hexdigest()

    print(f"Result:   {result}")
    print(f"Expected: {expected}")

    assert result == expected, "SHA-256 hash does not match expected value."
    print("SHA-256 hash matches expected value.")

test_hash()

Result:   031edd7d41651593c5fe5c006fa5752b37fddff7bc4e843aa6af0c950f4b9406
Expected: 031edd7d41651593c5fe5c006fa5752b37fddff7bc4e843aa6af0c950f4b9406
SHA-256 hash matches expected value.


## End of Problem 4

## Problem 5: Passwords

### Instruction

- The following are the SHA-256 hashes of three common passwords that have been hashed using one pass of the SHA-256 algorithm.
- As strings, they were encoded using [UTF-8](https://en.wikipedia.org/wiki/UTF-8).
- Determine the passwords and explain how you found them.
- Suggest ways in which the hashing of passwords could be improved to prevent the kind of attack you performed to find the passwords.

### SHA-256 Hashes

1. `5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8`
2. `873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34`
3. `b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342`

### Solution:

In [150]:
# The three SHA-256 hashes to crack
# Store them as hexadecimal strings in a list
target_hashes = [
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"
]

In [151]:
# Path to the file containing common passwords
password_file = "10_million_passwords.txt"

In [152]:
# Open the file and check each password
with open(password_file, "r", encoding="utf-8") as file:
    for line in file:
        candidate_password = line.strip()  # Remove newline characters and extra spaces

        # Compute the SHA-256 hash of the candidate password
        # See https://docs.python.org/3/library/hashlib.html
        password_hash = hashlib.sha256(candidate_password.encode("utf-8")).hexdigest()

        # Check if the computed hash matches any of the target hashes
        if password_hash in target_hashes:
            print(f"Found password: {candidate_password} -> {password_hash}")

Found password: password -> 5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8
Found password: cheese -> 873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34
Found password: P@ssw0rd -> b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342


## End of Problem 5

# End of the Assessment